In [47]:
import pandas as pd
import csv
import numpy as np

#訓練資料
train_df = pd.read_csv("./titanic_train.csv")
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [48]:
# 各變數中缺失值的比例
print(train_df.isnull().sum(axis = 0)/train_df.shape[0])

#增加 Has_Age欄位
train_df['Has_Age'] = train_df['Age'].isnull().map(lambda x : 0 if x == True else 1)
#增加 Has_Cabin欄位
train_df['Has_Cabin'] = train_df['Cabin'].isnull().map(lambda x : 0 if x == True else 1)
#Embarked遺失補值 "no"
train_df['Embarked']=train_df['Embarked'].fillna("No")
# Age欄位補值0
train_df['Age']=train_df['Age'].fillna(0)

print("----------------------------")

# 再次顯示各變數中缺失值的比例
print(train_df.isnull().sum(axis = 0)/train_df.shape[0])

print("----------------------------")

train_df.info()

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64
----------------------------
PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.000000
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.000000
Has_Age        0.000000
Has_Cabin      0.000000
dtype: float64
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass    

In [49]:
#資料EDA 

#Pclass vs Survived
display(train_df[["Pclass", "Survived"]].groupby(["Pclass"],as_index=False).mean().round(3))
#Sex vs Survive
display(train_df[["Sex", "Survived"]].groupby(["Sex"],as_index=False).mean().round(3))
#Age vs Survive
display(pd.pivot_table(train_df, values=['Age'], columns=['Survived'], aggfunc='mean'))
#Has_Age vs Survive
display(train_df[["Has_Age", "Survived"]].groupby(["Has_Age"],as_index=False).mean().round(3))
#SibSp vs Survive
display(pd.pivot_table(train_df, values=['SibSp'], columns=['Survived'], aggfunc='mean'))
#Parch vs Survive
display(pd.pivot_table(train_df, values=['Parch'], columns=['Survived'], aggfunc='mean'))
#Fare vs Survive
display(pd.pivot_table(train_df, values=['Fare'], columns=['Survived'], aggfunc='mean'))
#Has_Cabin vs Survive
display(train_df[["Has_Cabin", "Survived"]].groupby(["Has_Cabin"],as_index=False).mean().round(3))
#Embarkede vs Survive
display(train_df[["Embarked", "Survived"]].groupby(["Embarked"],as_index=False).mean().round(3))

,Pclass,Survived
0,1,0.630
1,2,0.473
2,3,0.242


,Sex,Survived
0,female,0.742
1,male,0.189


Survived,0,1
Age,23.653005,24.034123


,Has_Age,Survived
0,0,0.294
1,1,0.406


Survived,0,1
SibSp,0.553734,0.473684


Survived,0,1
Parch,0.32969,0.464912


Survived,0,1
Fare,22.117887,48.395408


,Has_Cabin,Survived
0,0,0.300
1,1,0.667


,Embarked,Survived
0,C,0.554
1,No,1.000
2,Q,0.390
3,S,0.337


In [50]:
#資料前處理 y=Survived x=Pclass, Sex, Age, SibSp, Parch, Fare, Embarked, Has_Age, Has_Cabin 
train_df_y = pd.DataFrame()
train_df_y["Survived"] = train_df['Survived'].astype(str)
train_df_y = train_df_y["Survived"].ravel()

fewvar_train_df_x=pd.DataFrame()
fewvar_train_df_x["Pclass"] = train_df['Pclass'].astype(str)
fewvar_train_df_x["Sex"] = train_df["Sex"].map(lambda x : 1 if x == "male" else 0) #1:男 0:女
fewvar_train_df_x["Age"] = train_df["Age"]
fewvar_train_df_x["SibSp"] = train_df["SibSp"]
fewvar_train_df_x["Parch"] = train_df["Parch"]
fewvar_train_df_x["Fare"] = train_df["Fare"]
fewvar_train_df_x["Embarked"] = train_df["Embarked"]
fewvar_train_df_x["Has_Age"] = train_df["Has_Age"]
fewvar_train_df_x["Has_Cabin"] = train_df["Has_Cabin"]

#轉dummy variable
train_df_x = pd.get_dummies(fewvar_train_df_x)

#min max scale
train_df_x=(train_df_x - train_df_x.min()) / (train_df_x.max()-train_df_x.min())
train_df_x

,Sex,Age,SibSp,Parch,Fare,Has_Age,Has_Cabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_No,Embarked_Q,Embarked_S
0,1.0,0.2750,0.125,0.000000,0.014151,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.4750,0.125,0.000000,0.139136,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.3250,0.000,0.000000,0.015469,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.4375,0.125,0.000000,0.103644,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.4375,0.000,0.000000,0.015713,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,1.0,0.3375,0.000,0.000000,0.025374,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
887,0.0,0.2375,0.000,0.000000,0.058556,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
888,0.0,0.0000,0.125,0.333333,0.045771,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
889,1.0,0.3250,0.000,0.000000,0.058556,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [52]:
#random forest tuning

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import  ensemble

rf_temp_score = 0
rf_tree_num = 0
for i in range(50,301,50):
    forest = ensemble.RandomForestClassifier(n_estimators = i) #n_estimators = 樹木數量
    scores = cross_val_score(forest,train_df_x,train_df_y,cv=5,scoring='accuracy')
    if scores.mean() > rf_temp_score :
        rf_temp_score = scores.mean()
        rf_tree_num = i
    print("ramdon forest","tree_num=",i)
    print(scores)
    print("accuracy:",scores.mean())

print("-----------------------------------------------------------------")
print("final_rf_tree_num=",rf_tree_num)
print("-----------------------------------------------------------------")

rf_temp_score = 0
rf_max_depth = 0
for i in range(3,11,1):
    forest = ensemble.RandomForestClassifier(n_estimators = rf_tree_num, max_depth = i) #n_estimators = 樹木數量
    scores = cross_val_score(forest,train_df_x,train_df_y,cv=5,scoring='accuracy')
    if scores.mean() > rf_temp_score :
        rf_temp_score = scores.mean()
        rf_max_depth = i
    print("ramdon forest","rf_max_depth=",i)
    print(scores)
    print("accuracy:",scores.mean())

print("-----------------------------------------------------------------")
print("final_rf_max_depth=",rf_max_depth)

ramdon forest tree_num= 50
[0.78212291 0.79775281 0.84831461 0.76966292 0.8258427 ]
accuracy: 0.8047391877471595
ramdon forest tree_num= 100
[0.75418994 0.78651685 0.85393258 0.75842697 0.83146067]
accuracy: 0.7969054045571526
ramdon forest tree_num= 150
[0.77094972 0.7752809  0.85955056 0.75280899 0.82022472]
accuracy: 0.7957629778419434
ramdon forest tree_num= 200
[0.75977654 0.79213483 0.84831461 0.75842697 0.81460674]
accuracy: 0.7946519364760529
ramdon forest tree_num= 250
[0.74860335 0.78089888 0.85393258 0.75280899 0.8258427 ]
accuracy: 0.7924172996045445
ramdon forest tree_num= 300
[0.75977654 0.78089888 0.84269663 0.75842697 0.8258427 ]
accuracy: 0.793528340970435
-----------------------------------------------------------------
final_rf_tree_num= 50
-----------------------------------------------------------------
ramdon forest rf_max_depth= 3
[0.7150838  0.75280899 0.85393258 0.79213483 0.80898876]
accuracy: 0.7845897934844015
ramdon forest rf_max_depth= 4
[0.7150838  0.8033

In [53]:
#KNN tuning

from sklearn.neighbors import KNeighborsClassifier

knn_temp_score = 0
knn_n_num = 0
for i in range(3,22,2):
    knn = KNeighborsClassifier(n_neighbors=i)
    scores = cross_val_score(knn,train_df_x,train_df_y,cv=5,scoring='accuracy')
    if scores.mean() > knn_temp_score :
        knn_temp_score = scores.mean()
        knn_n_num = i
    print("KNN","k=",i)
    print(scores)
    print("accuracy:",scores.mean())

print("-----------------------------------------------------------------")
print("final_knn_n_num=",knn_n_num)


KNN k= 3
[0.78212291 0.73595506 0.79775281 0.78089888 0.79775281]
accuracy: 0.7788964911179461
KNN k= 5
[0.75977654 0.7752809  0.81460674 0.80337079 0.78651685]
accuracy: 0.7879103634423451
KNN k= 7
[0.78212291 0.78089888 0.83707865 0.80337079 0.80898876]
accuracy: 0.8024919967359236
KNN k= 9
[0.78212291 0.81460674 0.81460674 0.79213483 0.81460674]
accuracy: 0.8036155922415416
KNN k= 11
[0.75977654 0.80898876 0.80898876 0.79775281 0.80337079]
accuracy: 0.795775531981671
KNN k= 13
[0.74860335 0.80898876 0.82022472 0.80898876 0.83707865]
accuracy: 0.8047768501663425
KNN k= 15
[0.75977654 0.80898876 0.81460674 0.80337079 0.83707865]
accuracy: 0.8047642960266147
KNN k= 17
[0.73743017 0.79775281 0.80898876 0.78651685 0.83707865]
accuracy: 0.7935534492498901
KNN k= 19
[0.74860335 0.78651685 0.80337079 0.78089888 0.82022472]
accuracy: 0.7879229175820728
KNN k= 21
[0.74860335 0.79213483 0.80898876 0.78651685 0.83707865]
accuracy: 0.7946644906157806
---------------------------------------------

In [54]:
#XGBoost tuning

from xgboost import XGBClassifier

xgb_temp_score = 0
xgb_tree_num = 0
for i in range(50,301,50):
    xgbc = XGBClassifier(eval_metric=['logloss','auc','error'], n_estimators=i)
    scores = cross_val_score(xgbc,train_df_x,train_df_y,cv=5,scoring='accuracy')
    if scores.mean() > xgb_temp_score :
        xgb_temp_score = scores.mean()
        xgb_tree_num = i 
    print("xgb","tree_num=",i)
    print(scores)
    print("accuracy:",scores.mean())
    
print("-----------------------------------------------------------------")
print("final_xgbc_tree_num=",xgb_tree_num)
print("-----------------------------------------------------------------")

xgb_temp_score = 0
xgb_max_depth = 0
for i in range(3,11):
    xgbc = XGBClassifier(eval_metric=['logloss','auc','error'], n_estimators=xgb_tree_num, max_depth=i)
    scores = cross_val_score(xgbc,train_df_x,train_df_y,cv=5,scoring='accuracy')
    if scores.mean() > xgb_temp_score :
        xgb_temp_score = scores.mean()
        xgb_max_depth = i 
    print("xgb","max_depth=",i)
    print(scores)
    print("accuracy:",scores.mean())
    
print("-----------------------------------------------------------------")
print("final_xgbc_max_depth=",xgb_max_depth)

C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


xgb tree_num= 50
[0.79329609 0.80337079 0.84831461 0.80898876 0.85393258]
accuracy: 0.8215805661917018


C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


xgb tree_num= 100
[0.77094972 0.80898876 0.84831461 0.78651685 0.85393258]
accuracy: 0.813740505931831


C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

xgb max_depth= 8
[0.7877095  0.80898876 0.86516854 0.79775281 0.85393258]
accuracy: 0.8227104387671835


C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


xgb max_depth= 9
[0.80446927 0.80337079 0.84269663 0.76966292 0.87078652]
accuracy: 0.8181972255351203


C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


xgb max_depth= 10
[0.77653631 0.80337079 0.85393258 0.78089888 0.86516854]
accuracy: 0.815981419873203
-----------------------------------------------------------------
final_xgbc_max_depth= 4


In [64]:
print("After Tuning Best Accuracy")
print("-----------------------------------------------------------------")
print("random forest:",rf_temp_score)
print("knn:",knn_temp_score)
print("xgboost:",xgb_temp_score)

After Tuning Best Accuracy
-----------------------------------------------------------------
random forest: 0.8238340342728016
knn: 0.8047768501663425
xgboost: 0.8372857949908982


In [59]:
#Build Model (choose xgboost)
forest = ensemble.RandomForestClassifier(n_estimators = rf_tree_num, max_depth = rf_max_depth)
forest_fit = forest.fit(train_df_x, train_df_y)

knn = KNeighborsClassifier(n_neighbors=knn_n_num)
knn_fit = knn.fit(train_df_x, train_df_y)

xgb = XGBClassifier(eval_metric=['logloss','auc','error'], n_estimators=xgb_tree_num, max_depth=xgb_max_depth)
xgb_fit = xgb.fit(train_df_x, train_df_y)


C:\Users\Admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [60]:
#測試資料
test_df = pd.read_csv("./titanic_test.csv")
#增加 Has_Age欄位
test_df['Has_Age'] = test_df['Age'].isnull().map(lambda x : 0 if x == True else 1)
#增加 Has_Cabin欄位
test_df['Has_Cabin'] = test_df['Cabin'].isnull().map(lambda x : 0 if x == True else 1)
#Embarked遺失補值 "no"
test_df['Embarked']=test_df['Embarked'].fillna("No")
# Age欄位補值0
test_df['Age']=test_df['Age'].fillna(0)

#test_data取出用到變數
fewvar_test_df_x=pd.DataFrame()
fewvar_test_df_x["Pclass"] = test_df['Pclass'].astype(str)
fewvar_test_df_x["Sex"] = test_df["Sex"].map(lambda x : 1 if x == "male" else 0) #1:男 0:女
fewvar_test_df_x["Age"] = test_df["Age"]
fewvar_test_df_x["SibSp"] = test_df["SibSp"]
fewvar_test_df_x["Parch"] = test_df["Parch"]
fewvar_test_df_x["Fare"] = test_df["Fare"]
fewvar_test_df_x["Embarked"] = test_df["Embarked"]
fewvar_test_df_x["Has_Age"] = test_df["Has_Age"]
fewvar_test_df_x["Has_Cabin"] = test_df["Has_Cabin"]

#轉dummy variable
test_df_x = pd.get_dummies(fewvar_test_df_x)

#min max scale
test_df_x=(test_df_x - test_df_x.min()) / (test_df_x.max()-test_df_x.min())
#沒有Embarked_No所以補一欄
test_df_x["Embarked_No"] = 0
#測試集Fare平均插值
test_df_x['Fare'] = test_df_x['Fare'].fillna(test_df_x['Fare'].mean())
test_df_x

,Sex,Age,SibSp,Parch,Fare,Has_Age,Has_Cabin,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Embarked_No
0,1.0,0.453947,0.000,0.000000,0.015282,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
1,0.0,0.618421,0.125,0.000000,0.013663,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
2,1.0,0.815789,0.000,0.000000,0.018909,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
3,1.0,0.355263,0.000,0.000000,0.016908,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
4,0.0,0.289474,0.125,0.111111,0.023984,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1.0,0.000000,0.000,0.000000,0.015713,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
414,0.0,0.513158,0.000,0.000000,0.212559,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0
415,1.0,0.506579,0.000,0.000000,0.014151,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
416,1.0,0.000000,0.000,0.000000,0.015713,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0


In [176]:
# random forest submit
rf_res =  forest_fit.predict(test_df_x)
submit = pd.DataFrame()
submit['PassengerId'] = test_df['PassengerId']
submit['Survived'] = rf_res
submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit.csv', index= False)

In [61]:
# xgboost submit (priority)
xgb_res =  xgb_fit.predict(test_df_x)
submit = pd.DataFrame()
submit['PassengerId'] = test_df['PassengerId']
submit['Survived'] = xgb_res
submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit.csv', index= False)

In [62]:
submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
